#Indexação do arquivo de telegramas do Wikileaks

Nesta prática vamos indexar todo oarquivo de telegramas do Wikileaks com o elasticsearch e testar algumas de suas características como sistema de recuperação de informação.

Para realizar esta prática, é necessário fazer o Download do seguinte arquivo:
https://archive.org/details/wikileaks-cables-csv

Também é necessário instalar a biblioteca cablemap: https://github.com/heuer/cablemap


In [11]:
from cablemap.core import cables_from_source
from elasticsearch import Elasticsearch as ES
from elasticsearch import helpers


Primeiro temos que localizar o arquivo com os telegramas.

In [6]:
fname = "../../../../Downloads/wikileaks_cables/cables.csv"

In [8]:
i = 0
for cable in cables_from_source(fname):
    print(cable.subject, cable.created)
    if i>5:
        break
    i += 1

(u'EXTENDED NATIONAL JURISDICTIONS OVER HIGH SEAS', u'1966-12-28 18:48')
(u'ACCELERATION OF F-4ES FOR IRAN', u'1972-02-25 09:30')
(u'TRIALS/EXECUTIONS OF ANTI-GOVERNMENT ELEMENTS: STUDENTS DEMONSTRATE AND SHAH LASHES OUT AT FOREIGN CRITICS', u'1972-03-09 05:40')
(u'CONTINUING TERRORIST ACTIVITIES IN IRAN', u'1972-08-10 04:00')
(u'CONTINUING TERRORIST VIOLENCE', u'1972-08-22 09:27')
(u'AUDIENCE WITH SHAH APRIL 5', u'1973-04-02 08:34')
(u'ASSASSINATION/KIDNAP PLOT AGAINST SHAH REVEALED', u'1973-10-02 14:00')


Cada telegrama tem vários atributos, por isso precisamos construir um documento JSON com todas estes atributos

Vamos ignorar dois atributos por causa de bugs na biblioteca cablemap

In [32]:
atributos = [i for i in dir(cable) if not i.startswith('_')]
print atributos
atributos.pop(3) #remove "classification_categories"
atributos.pop(4) #remove "comment"
print atributos

['cabledrum_uri', 'canonical_id', 'classification', 'classification_categories', 'classified_by', 'comment', 'content', 'created', 'header', 'info_recipients', 'is_partial', 'nondisclosure_deadline', 'origin', 'plusd_canonical_id', 'plusd_uri', 'recipients', 'reference_id', 'references', 'released', 'signed_by', 'subject', 'summary', 'tags', 'transmission_id', 'wl_uris']
['cabledrum_uri', 'canonical_id', 'classification', 'classified_by', 'content', 'created', 'header', 'info_recipients', 'is_partial', 'nondisclosure_deadline', 'origin', 'plusd_canonical_id', 'plusd_uri', 'recipients', 'reference_id', 'references', 'released', 'signed_by', 'subject', 'summary', 'tags', 'transmission_id', 'wl_uris']


In [33]:
getattr(cable,'cabledrum_uri')

u'http://www.cabledrum.net/cables/73TEHRAN7005'

Agora temos que configurar o Elasticsearch para receber a nossa coleção:

In [34]:
es = ES()
es.indices.create(index='wikileaks', ignore=400)

{u'error': u'IndexAlreadyExistsException[[wikileaks] already exists]',
 u'status': 400}

In [ ]:

def cable_doc_gen():
    for j,cable in enumerate(cables_from_source(fname)):
        try:
            doc = {a: getattr(cable, a) for a in atributos}
        except AttributeError as e:
            print e
            continue
        action = {
            "_index": "wikileaks",
            "_type": "telegramas",
            "_id": j,
            "doc": doc
            }
        if j%100 == 0:
            print "Indexando telegrama número {}".format(j)
        yield action
        
helpers.bulk(es, cable_doc_gen())

Indexando telegrama no 0
Indexando telegrama no 100

FM SECSTATE WASHDC
INFO AMEMBASSY ATHENS
"
FM SECSTATE WASHDC
INFO USDEL SECRETARY IMMEDIATE
"



Indexando telegrama no 200

FM SECSTATE WASHDC
INFO USMISSION GENEVA IMMEDIATE
AMEMBASSY HELSINKI IMMEDIATE
"



Indexando telegrama no 300

...


'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama no 97300
Indexando telegrama no 97400
Indexando telegrama no 97500
Indexando telegrama no 97600
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama no 97700
Indexando telegrama no 97800
'Reference' object has no attribute 'name'
Indexando telegrama no 97900
Indexando telegrama no 98000
'Reference' object has no attribute 'name'
Indexando telegrama no 98100
Indexando telegrama no 98200
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama no 98300

RR RUEHWEB

DE RUEHLM #0333/01 0590526
ZNR UUUUU ZZH
R 280526Z FEB 07
FM AMEMBASSY COLOMBO
INFO SECSTATE WASHDC 5532"



'Reference' object has no attribute 'name'
Indexando telegrama no 98400
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama no 98600
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama no 98700
Indexando telegrama no 98800
Indexando telegrama no 98900
'Reference' object has no attribute 'name'
Indexando telegrama no 99000
Indexando telegrama no 99100
Indexando telegrama no 99200
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama no 99300
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama no 99400
Indexando telegrama no 99500
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
'Reference' object has no attribute 'name'
Indexando telegrama no 99600
Indexando telegrama no 99700
Indexando telegrama no 99800
'Reference' object has no attribut

PP RUEHPB
DE RUEHDT #0115/01 0791112
ZNR UUUUU ZZH
P 201112Z MAR 07
FM AMEMBASSY DILI
INFO RUEHDT/AMEMBASSY DILI 2735
RUCNDT/USMISSION USUN NEW YORK 0874
RUCNARF/ASEAN REGIONAL FORUM COLLECTIVE
RHEHAAA/NATIONAL SECURITY COUNCIL WASHINGTON DC
RHHMUNA/USPACOM HONOLULU HI
RUEHBY/AMEMBASSY CANBERRA PRIORITY 0933
RUEHLI/AMEMBASSY LISBON PRIORITY 0824
RUEHLO/AMEMBASSY LONDON PRIORITY 0632
RUEHKO/AMEMBASSY TOKYO PRIORITY 0675
RUEHWL/AMEMBASSY WELLINGTON PRIORITY 0759
RUEHBR/AMEMBASSY BRASILIA PRIORITY 0483
RHEFDIA/DIA WASHINGTON DC
RUEAIIA/CIA WASHDC
RUEHROV/AMEMBASSY VATICAN 0069"



'Reference' object has no attribute 'name'
Indexando telegrama no 101200
Indexando telegrama no 101300
Indexando telegrama no 101400
Indexando telegrama no 101500
Indexando telegrama no 101600
Indexando telegrama no 101700
Indexando telegrama no 101800
Indexando telegrama no 101900

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__implemented__',
 '__init__',
 '__module__',
 '__new__',
 '__providedBy__',
 '__provides__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 'cabledrum_uri',
 'canonical_id',
 'classification',
 'classification_categories',
 'classified_by',
 'comment',
 'content',
 'created',
 'header',
 'info_recipients',
 'is_partial',
 'nondisclosure_deadline',
 'origin',
 'plusd_canonical_id',
 'plusd_uri',
 'recipients',
 'reference_id',
 'references',
 'released',
 'signed_by',
 'subject',
 'summary',
 'tags',
 'transmission_id',
 'wl_uris']

AttributeError: 'module' object has no attribute 'parse_classification_categories'

cabledrum_uri
http://www.cabledrum.net/cables/73TEHRAN7005
canonical_id
73TEHRAN7005
classification
UNCLASSIFIED
classified_by
()
content
UNCLAS TEHRAN 7005 
 
Declassified/Released US Department of State EO Systematic Review 30 JUN 2005 
 
E.O. 11652: N/A 
TAGS: PINS IR 
SUBJECT: ASSASSINATION/KIDNAP PLOT AGAINST SHAH REVEALED 
 
SUMMARY: GOI ANNOUNCED TODAY ARREST OF TWELVE PERSONS 
INCLUDING TWO WOMEN FOR PLOTTING TO KIDNAP OR KILL SHAH, 
EMPRESS AND OTHER MEMBERS OF IMPERIAL FAMILY. PLOTTERS 
SAID TO BELONG TO WING OF OUTLAWED TUDEH (COMMUNIST) PARTY 
AND ARE SAID TO HAVE MADE CONFESSIONS. END SUMMARY. 
 
1. MINISTRY OF INFORMATION ANNOUNCED OCTOBER 2 THE ARREST OF 
TWELVE PERSONS INCLUDING TWO WOMEN ON CHARGES OF PLOTTING TO 
KIDNAP OR KILL MEMBERS OF THE IMPERIAL FAMILY. ACCORDING TO 
OFFICIAL ANNOUNCEMENT, GROUP, WHICH INCLUDED FILMMAKERS, CAMERAMEN 
AND NEWSPAPERMEN, HAD RECONNOITERED SHAH'S CASPIAN PALACE 
AT NOWSHAHR AS WELL AS RESIDENCE OF HIM'S YOUNGER SISTER 
PRINCESS FATE

['cabledrum_uri',
 'canonical_id',
 'classification',
 'classification_categories',
 'classified_by',
 'comment',
 'content',
 'created',
 'header',
 'info_recipients',
 'is_partial',
 'nondisclosure_deadline',
 'origin',
 'plusd_canonical_id',
 'plusd_uri',
 'recipients',
 'reference_id',
 'references',
 'released',
 'signed_by',
 'subject',
 'summary',
 'tags',
 'transmission_id',
 'wl_uris']

u"UNCLAS TEHRAN 7005 \n \nDeclassified/Released US Department of State EO Systematic Review 30 JUN 2005 \n \nE.O. 11652: N/A \nTAGS: PINS IR \nSUBJECT: ASSASSINATION/KIDNAP PLOT AGAINST SHAH REVEALED \n \nSUMMARY: GOI ANNOUNCED TODAY ARREST OF TWELVE PERSONS \nINCLUDING TWO WOMEN FOR PLOTTING TO KIDNAP OR KILL SHAH, \nEMPRESS AND OTHER MEMBERS OF IMPERIAL FAMILY. PLOTTERS \nSAID TO BELONG TO WING OF OUTLAWED TUDEH (COMMUNIST) PARTY \nAND ARE SAID TO HAVE MADE CONFESSIONS. END SUMMARY. \n \n1. MINISTRY OF INFORMATION ANNOUNCED OCTOBER 2 THE ARREST OF \nTWELVE PERSONS INCLUDING TWO WOMEN ON CHARGES OF PLOTTING TO \nKIDNAP OR KILL MEMBERS OF THE IMPERIAL FAMILY. ACCORDING TO \nOFFICIAL ANNOUNCEMENT, GROUP, WHICH INCLUDED FILMMAKERS, CAMERAMEN \nAND NEWSPAPERMEN, HAD RECONNOITERED SHAH'S CASPIAN PALACE \nAT NOWSHAHR AS WELL AS RESIDENCE OF HIM'S YOUNGER SISTER \nPRINCESS FATEMEH. PLAN WAS TO KILL SHAH, EMPRESS, CROWN \nPRINCE AND POSSIBLY OTHERS, PERHAPS INCLUDING UNNAMED FOREIGN \nAMBASSA